# Analysing Smartphone Prices

In [1]:
import requests
from urllib import parse
import re

import pandas as pd
from tqdm import tqdm

https://celulares.mercadolivre.com.br/Galaxy%20TabS%2010.5


https://celulares.mercadolivre.com.br/Galaxy%20TabS%208.4
(' Sansung Tab Pro T320 8.4 Res 2500x1600p 32gb Sd Classe 10 ',)
('800',)


https://celulares.mercadolivre.com.br/Galaxy%20TabS3
(' Samsung Galaxy Tab3 ',)
('305',)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_original_name   3 non-null      object
 1   item_extracted_name  2 non-null      object
 2   item_price           2 non-null      object
dtypes: object(3)
memory usage: 200.0+ bytes
None


## 1. Loading Devices Names

In [14]:
RELATIVE_PATH_TO_DATA_FOLDER = './data/'
filename = 'devices_using_ribonapp_data'
filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + filename + '.csv'
devices_df = pd.read_csv(filename_with_path, skiprows=0, names=['model_name', 'old_price_range'])
devices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MODEL_NAME       1130 non-null   object
 1   OLD_PRICE_RANGE  1130 non-null   object
dtypes: object(2)
memory usage: 17.8+ KB


### 1.1 Cleaning Duplicates and Missing Values

In [19]:
devices_df.drop_duplicates(['model_name'], inplace=True)
devices_df.dropna(axis=0, inplace=True)
devices_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130 entries, 0 to 1129
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MODEL_NAME       1130 non-null   object
 1   OLD_PRICE_RANGE  1130 non-null   object
dtypes: object(2)
memory usage: 26.5+ KB


## 2. Searching for Prices

In [22]:
url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()

url_list = [url_base + parse.quote(item_name) for item_name in item_list]
item_list_tuple = []

for url, item_original_name in tdqm(zip(url_list, item_list)):
    r = requests.get(url)
    text_content = str(r.text)
    item_name = re.search(r'class="main-title">([\s\S]+?)<\/span>', text_content)
    item_price = re.search(r'price__fraction">(\S+)<\/span>', text_content)
    if item_name:
        item_name = item_name.groups()[0]
    if item_price:
        item_price = item_price.groups()[0]
    item_list_tuple.append((item_original_name, item_name, item_price))


extracted_data_df = pd.DataFrame(data=item_list_tuple, columns=['item_original_name','item_extracted_name', 'item_price'])
print(extracted_data.info())

https://celulares.mercadolivre.com.br/deviceModel


https://celulares.mercadolivre.com.br/2%20A2005


https://celulares.mercadolivre.com.br/3
(' Samsung Galaxy A30s Dual SIM 64 GB Prism crush white 4 GB RAM ',)
('1.111',)


https://celulares.mercadolivre.com.br/3T
(' Celular Oneplus 3t Dual A3010 4g 64gb 5.5 16mp/16mp Cinza ',)
('2.400',)


https://celulares.mercadolivre.com.br/4047D


https://celulares.mercadolivre.com.br/5
(' Samsung Galaxy J2 Core Dual SIM 16 GB Prata (Lavanda) 1 GB RAM ',)
('519',)


https://celulares.mercadolivre.com.br/5051J
(' Celular Alcatel Pop 4 5051j ',)
('239',)


https://celulares.mercadolivre.com.br/5056D


https://celulares.mercadolivre.com.br/5056E


https://celulares.mercadolivre.com.br/5T
(' Huawei Nova Series Nova 5t Dual SIM 128 GB Black 8 GB RAM ',)
('2.800',)


https://celulares.mercadolivre.com.br/6
(' Samsung Galaxy J4 Core Dual SIM 16 GB Preto 1 GB RAM ',)
('619',)


https://celulares.mercadolivre.com.br/6039J
(' Alcatel Idol 3 6039 Dual 4.7 4g



https://celulares.mercadolivre.com.br/BLADE%20A610


https://celulares.mercadolivre.com.br/BLADE%20L110
(' Celular Zte Blade L110 Defeito (ler Descricao) 2/19 ',)
('50',)


https://celulares.mercadolivre.com.br/BLADE%20V0800


https://celulares.mercadolivre.com.br/BLU%20Dash%20XL
(' Smartphone Blu Dash Xl {d710l} ',)
('449',)


https://celulares.mercadolivre.com.br/BLU%20Grand%20M
(' BLU Z5 Dual SIM 32 MB Cinza 32 MB RAM ',)
('92',)


https://celulares.mercadolivre.com.br/BLU%20S1
(' BLU Zoey Flex Dual SIM 32 MB Dourado 32 MB RAM ',)
('550',)


https://celulares.mercadolivre.com.br/BLU%20STUDIO%20J8%20LTE
(' Celular Blu Studio J8 Lte Preço Negociável ',)
('190',)


https://celulares.mercadolivre.com.br/BLU%20VIVO%20XL2
(' Motorola Moto Z Z2 Play Dual SIM 64 GB Ouro-fino 4 GB RAM ',)
('1.800',)


https://celulares.mercadolivre.com.br/BLU%20Vivo%205%20Mini
(' Celular Touch Blu 5 Mini ',)
('195',)


https://celulares.mercadolivre.com.br/BLade%20Zmax


https://celulares.mercadolivre.com.

(' Galaxy S3 Neo I9300i Para Retirar Peças Ref X14 ',)
('30',)


https://celulares.mercadolivre.com.br/Galaxy%20A10
(' Samsung Galaxy A10 Dual SIM 32 GB Preto 2 GB RAM ',)
('819',)


https://celulares.mercadolivre.com.br/Galaxy%20A10e
(' Samsung Galaxy 10.1 ',)
('800',)


https://celulares.mercadolivre.com.br/Galaxy%20A20
(' Samsung Galaxy A20 Dual SIM 32 GB Preto 3 GB RAM ',)
('1.099',)


https://celulares.mercadolivre.com.br/Galaxy%20A20e


https://celulares.mercadolivre.com.br/Galaxy%20A3
(' Samsung Galaxy A3 Dual SIM 16 GB Preto-meia-noite 1 GB RAM ',)
('699',)


https://celulares.mercadolivre.com.br/Galaxy%20A3%20Duos
(' Samsung Galaxy A3 Dual SIM 16 GB Preto-meia-noite 1 GB RAM ',)
('699',)


https://celulares.mercadolivre.com.br/Galaxy%20A3%282016%29
(' Celular Samsung Galaxy A3 2016 Usado Seminovo Muito Bom ',)
('419',)


https://celulares.mercadolivre.com.br/Galaxy%20A3%282017%29
(' Celular ',)
('330',)


https://celulares.mercadolivre.com.br/Galaxy%20A30
(' Samsung Galaxy A30

(' Samsung Galaxy J2 Core Dual SIM 16 GB Prata (Lavanda) 1 GB RAM ',)
('519',)


https://celulares.mercadolivre.com.br/Galaxy%20J2%20Core
(' Samsung Galaxy J2 Core Dual SIM 16 GB Prata (Lavanda) 1 GB RAM ',)
('519',)


https://celulares.mercadolivre.com.br/Galaxy%20J2%20Prime
(' Samsung Galaxy J2 Prime Dual SIM 16 GB Preto 1.5 GB RAM ',)
('748',)


https://celulares.mercadolivre.com.br/Galaxy%20J2%20Pro
(' Celular Samsung Galaxy J2 Pro Usado Seminovo Mt Bom ',)
('439',)


https://celulares.mercadolivre.com.br/Galaxy%20J3
(' Samsung Galaxy J3 (2016) Dual SIM 8 GB Dourado 1.5 GB RAM ',)
('579',)


https://celulares.mercadolivre.com.br/Galaxy%20J3%20Pop
(' Samsung Galaxy J3 (2016) Dual SIM 8 GB Dourado 1.5 GB RAM ',)
('579',)


https://celulares.mercadolivre.com.br/Galaxy%20J3%20Pro
(' Celular J3 ',)
('80',)


https://celulares.mercadolivre.com.br/Galaxy%20J3%282016%29
(' Samsung Galaxy J3 (2016) Dual SIM 8 GB Dourado 1.5 GB RAM ',)
('579',)


https://celulares.mercadolivre.com.br/Galaxy%

(' Celular Samsung S 5310 Para Retirar Peças Os 10110 ',)
('39',)


https://celulares.mercadolivre.com.br/Galaxy%20Pocket%20Neo%20S5310
(' Samsung Galaxy Pocket Neo S5310 3g Android 4.1 - Novo ',)
('329',)


https://celulares.mercadolivre.com.br/Galaxy%20S%205%20%28T-Mobile%29%20%5BSM-G900T%5D


https://celulares.mercadolivre.com.br/Galaxy%20S%20Advance%20%28GT-i9070/GT-i9070P%29
(' Samsung Galaxy A20s 32 GB Preto 3 GB RAM ',)
('1.233',)


https://celulares.mercadolivre.com.br/Galaxy%20S%20II%20%28GT-I9100%29
(' Smartphone Samsung Galaxy S2 Com 16gb De Memória Preto ',)
('451',)


https://celulares.mercadolivre.com.br/Galaxy%20S%20II%20GT-S7273T
(' Celular Samsung Unico Dono Galaxy S Ii 2 Tv Compl Impec Caix ',)
('320',)


https://celulares.mercadolivre.com.br/Galaxy%20S%20III
(' Samsung Galaxy S III Mini 8 GB Pebble blue 1 GB RAM ',)
('449',)


https://celulares.mercadolivre.com.br/Galaxy%20S%20III%20mini%20%28GT-I8200L%29
(' Samsung Galaxy S3 Mini Gt-i8200l (com Defeito) ',)
('90',)


...

(' Carregador Turbo . ',)
('70',)


https://celulares.mercadolivre.com.br/Smart%20ultra%207


https://celulares.mercadolivre.com.br/Smart_N9_Lite
(' Huawei P Series P Smart 2019 Dual SIM 64 GB Azul-aurora 3 GB RAM ',)
('1.669',)


https://celulares.mercadolivre.com.br/Smarti%20ANDROID%20S420
(' Samsung Galaxy S9 Dual SIM 128 GB Cinza-titânio 4 GB RAM ',)
('2.288',)


https://celulares.mercadolivre.com.br/Spirit%204G%20MS870
(' Lg Spirit 3g H420 Usado! ',)
('360',)


https://celulares.mercadolivre.com.br/Spree
(' Lg K4 K120 (com Defeito) ',)
('70',)


https://celulares.mercadolivre.com.br/Starnaute%204


https://celulares.mercadolivre.com.br/Studio%20C%20HD
(' Celular Blu Studio C Hd S-090q - 5.0 - Dual-sim - 8gb - 3g ',)
('339',)


https://celulares.mercadolivre.com.br/Studio%20J2
(' Celular Blu Grand Studio J2 Lacrado Anatel Tela5 Android Nfe ',)
('599',)


https://celulares.mercadolivre.com.br/Studio%20View%20XL
(' Smartphone Blu Studio View Xl Dual Sim 3g 5.7 Garantia 1 Ano ',)
('59

(' Sony Xperia XZ Premium Dual SIM 64 GB Preto-mar-profundo 4 GB RAM ',)
('3.760',)


https://celulares.mercadolivre.com.br/X%20cam
(' Samsung Galaxy A30s Dual SIM 64 GB Prism crush white 4 GB RAM ',)
('1.111',)


https://celulares.mercadolivre.com.br/X%20max
(' Asus ZenFone Max Shot ZB634KL Dual SIM 64 GB Preto 4 GB RAM ',)
('945',)


https://celulares.mercadolivre.com.br/X%20power
(' Motorola Moto G G7 Power Dual SIM 64 GB Ceramic black 4 GB RAM ',)
('1.495',)


https://celulares.mercadolivre.com.br/X%20screen
(' Xiaomi Redmi 7A (13 Mpx) Dual SIM 16 GB Morning blue 2 GB RAM ',)
('678',)


https://celulares.mercadolivre.com.br/X008D


https://celulares.mercadolivre.com.br/X1%20Lite
(' Neffos X1 Lite ',)
('649',)


https://celulares.mercadolivre.com.br/X19
(' Cubot X19 ',)
('800',)


https://celulares.mercadolivre.com.br/X30
(' Celular ',)
('699',)


https://celulares.mercadolivre.com.br/X5%20max%20pro
(' Smartphone Doogee X5 Max Pro ',)
('400',)


https://celulares.mercadolivre.com.br

(' Huawei Y340-u081 Preto ',)
('450',)


https://celulares.mercadolivre.com.br/Y5
(' Celular Huawei Honor Y5ll Cun-u29 Dual 8gb Original Vitrine ',)
('558',)


https://celulares.mercadolivre.com.br/Y5II


https://celulares.mercadolivre.com.br/Y6
(' Celular Huawei Y6 2018 16gb 2gb Azul + Nf ',)
('949',)


https://celulares.mercadolivre.com.br/Y6%202017
(' Samsung Galaxy A5 (2017) 32 GB Preto 3 GB RAM ',)
('1.250',)


https://celulares.mercadolivre.com.br/Y6%202019


https://celulares.mercadolivre.com.br/Y6%20Max
(' Celular Huawei Y6 2018 16gb 2gb Azul + Nf ',)
('1.011',)


https://celulares.mercadolivre.com.br/Y7
(' Smartphone Huawei Y7 Pro - Global 4g + 64gb Azul ',)
('1.590',)


https://celulares.mercadolivre.com.br/Y7Plus


https://celulares.mercadolivre.com.br/Y8
(' Smartphone Doogee Y8 32gb 3gb Ram Verde ',)
('799',)


https://celulares.mercadolivre.com.br/Y9%20Prime%202019
(' Huawei Y9 Prime Dual 128gb - Câmera Pop Up - Completo + Case ',)
('1.500',)


https://celulares.mercadoliv



https://celulares.mercadolivre.com.br/iPad%203rd%20Gen%20%28Wi-Fi%20Only%29


https://celulares.mercadolivre.com.br/iPad%203rd%20Gen%20%28Wi-Fi/4G%20LTE%20Verizon/GPS%29


https://celulares.mercadolivre.com.br/iPad%204th%20Gen%20%28Wi-Fi%20Only%29


https://celulares.mercadolivre.com.br/iPad%204th%20Gen%20%28Wi-Fi/4G%20LTE%20AT%26T/GPS%29


https://celulares.mercadolivre.com.br/iPad%206th%20Generation


https://celulares.mercadolivre.com.br/iPad%209.7-Inch%205th%20Gen%20%28Cellular%29


https://celulares.mercadolivre.com.br/iPad%209.7-Inch%205th%20Gen%20%28WIFI%29


https://celulares.mercadolivre.com.br/iPad%20Air%20%28Wi-Fi%20Only%29
(' iPad Mini 3 Wifi 64gb - A1599 ',)
('1.500',)


https://celulares.mercadolivre.com.br/iPad%20Air%20%28iPad4%2C2%29
(' Apple iPhone 5c 8 GB Branco 1 GB RAM ',)
('849',)


https://celulares.mercadolivre.com.br/iPad%20Air%202%20-%20Cellular


https://celulares.mercadolivre.com.br/iPad%20Air%202%20-%20Wi-fi


https://celulares.mercadolivre.com.br/iPad%20M

(' Samsung Galaxy J2 Core Dual SIM 16 GB Prata (Lavanda) 1 GB RAM ',)
('519',)


https://celulares.mercadolivre.com.br/t1_8p0lte
(' Celular Motorola V60i Não É Nem Usa Chip. Enviamos T.brasil ',)
('75',)


https://celulares.mercadolivre.com.br/vivo%201724
(' Multilaser Flip Vita Dual SIM 32 MB Azul/Preto 32 MB RAM ',)
('167',)


https://celulares.mercadolivre.com.br/vivo%20X9
(' Multilaser Flip Vita Dual SIM 32 MB Azul/Preto 32 MB RAM ',)
('167',)


https://celulares.mercadolivre.com.br/vivo%20Y53
(' Multilaser Flip Vita Dual SIM 32 MB Azul/Preto 32 MB RAM ',)
('167',)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_original_name   1130 non-null   object
 1   item_extracted_name  903 non-null    object
 2   item_price           903 non-null    object
dtypes: object(3)
memory usage: 26.6+ KB
None


### 2.1 Treating Data

In [42]:

extracted_data.dropna(axis=0, inplace=True)
item_raw_prices_list = extracted_data['item_price'].to_list()
item_treated_prices_list = []
for item_raw_price in item_raw_prices_list:
    item_price = None
    if '.' in item_raw_price:
        thousands_digits, hundreds_digits = item_raw_price.split('.')
        item_price = int(thousands_digits)*1000 + int(hundreds_digits)
    else:
        item_price = int(item_raw_price)
    item_treated_prices_list.append(item_price)
extracted_data['item_price'] = item_treated_prices_list
extracted_data

,item_original_name,item_extracted_name,item_price
2,3,Samsung Galaxy A30s Dual SIM 64 GB Prism crus...,1111
3,3T,Celular Oneplus 3t Dual A3010 4g 64gb 5.5 16m...,2400
5,5,Samsung Galaxy J2 Core Dual SIM 16 GB Prata (...,519
6,5051J,Celular Alcatel Pop 4 5051j,239
9,5T,Huawei Nova Series Nova 5t Dual SIM 128 GB Bl...,2800
...,...,...,...
1125,nova 4,Samsung Galaxy J2 Core Dual SIM 16 GB Prata (...,519
1126,t1_8p0lte,Celular Motorola V60i Não É Nem Usa Chip. Env...,75
1127,vivo 1724,Multilaser Flip Vita Dual SIM 32 MB Azul/Pret...,167
1128,vivo X9,Multilaser Flip Vita Dual SIM 32 MB Azul/Pret...,167


### 2.1 Storing Found Prices

In [43]:
extracted_filename = 'extracted_device_data'
extracted_filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + extracted_filename + '.csv'

In [45]:
extracted_data_df.to_csv(extracted_filename_with_path, index=False)

## 3. Analysing Prices Ranges

In [46]:
device_extracted_data_df = pd.read_csv(extracted_filename_with_path, header=0)
device_extracted_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_original_name   903 non-null    object
 1   item_extracted_name  903 non-null    object
 2   item_price           903 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.3+ KB


In [47]:
device_extracted_data_df.head()

,item_original_name,item_extracted_name,item_price
0,3,Samsung Galaxy A30s Dual SIM 64 GB Prism crus...,1111
1,3T,Celular Oneplus 3t Dual A3010 4g 64gb 5.5 16m...,2400
2,5,Samsung Galaxy J2 Core Dual SIM 16 GB Prata (...,519
3,5051J,Celular Alcatel Pop 4 5051j,239
4,5T,Huawei Nova Series Nova 5t Dual SIM 128 GB Bl...,2800


In [48]:
extracted_data.head(10)

,item_original_name,item_extracted_name,item_price
2,3,Samsung Galaxy A30s Dual SIM 64 GB Prism crus...,1111
3,3T,Celular Oneplus 3t Dual A3010 4g 64gb 5.5 16m...,2400
5,5,Samsung Galaxy J2 Core Dual SIM 16 GB Prata (...,519
6,5051J,Celular Alcatel Pop 4 5051j,239
9,5T,Huawei Nova Series Nova 5t Dual SIM 128 GB Bl...,2800
10,6,Samsung Galaxy J4 Core Dual SIM 16 GB Preto 1...,619
11,6039J,Alcatel Idol 3 6039 Dual 4.7 4g 13mp 16gb Ana...,629
13,6055B,Smartphone Alcatel Idol 4 6055b 16gb 3gb Ram ...,549
14,62S,Celular 62s Mirage,599
15,6T,Smartphone Oneplus 6t Mclaren 10gb/256gb Ediç...,3300
